In [ ]:
import torch
import librosa
import evaluate
import numpy as np
import gradio as gr
import librosa.display
import matplotlib.pyplot as plt

from torch import nn, optim
from torch.utils.data import DataLoader
from dataclasses import dataclass, field
from datasets import load_dataset, Audio
from typing import Any, Dict, List, Optional, Union
from transformers import AutoProcessor, AutoModelForCTC, TrainingArguments, Trainer, pipeline



In [44]:
from src.data.dataset import CommonVoice
from src.data.vocab import SimpleVocab
from src.training.early_stopping import CustomEarlyStopping
from src.training.train import train_model
from src.training.evaluate import evaluate_model

In [49]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

In [2]:
ds_train = CommonVoice("train")
ds_val = CommonVoice("validation")
ds_test = CommonVoice("test")

/Users/fcisternas/miniconda3/envs/bats/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [63]:
x.shape

(13, 200)

In [65]:
y.shape[0]

15

In [50]:
#prepare data for training
def collate_fn(batch):
    x, y = zip(*batch)
    x = nn.utils.rnn.pad_sequence(x, batch_first=True)
    y = nn.utils.rnn.pad_sequence(y, batch_first=True)
    return x, y

batch_size = 32
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
dl_val = DataLoader(ds_val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

#create pytorch model


In [62]:
for x,y in dl_train:
    print(x.shape)
    print(y.shape)
    break

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [57]:
y.shape[0]

15

In [58]:
# create pytorch model

class BaselineModel(nn.Module):
    def __init__(self, output_size, hidden_size=128, num_layers=2, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=40,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout,
        )
        self.linear = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [59]:
# train model
model = BaselineModel(output_size=y.shape[0])
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
early_stopping = CustomEarlyStopping(patience=5, verbose=True)

In [ ]:
# traning loop from scratch




In [51]:
# Hiperparametros del modelo
input_size = x.shape[1]
hidden_size = 128
num_layers = 2
dropout = 0.2
bidirectional = True
layer_tipe = nn.LSTM
output_size = y.shape[0]
batch_size = x.shape[0]
criterion = nn.CrossEntropyLoss()
epochs = 10

#custom early stopping
early_stopping = CustomEarlyStopping(patience=3, verbose=True)

# Para RNN simple
model = RNNModel(input_size, hidden_size, output_size,num_layers, dropout, bidirectional, layer_tipe)

#optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.001)

In [60]:
import torch
from src.training.early_stopping import CustomEarlyStopping

def train_model(model: torch.nn.Module, train_loader: torch.utils.data.DataLoader, val_loader: torch.utils.data.DataLoader, epochs: int, criterion: torch.nn, optimizer: torch.optim, early_stopping: CustomEarlyStopping = None):
    """
    Trains a PyTorch model with the given data loaders, criterion, and optimizer.

    Args:
        model: The PyTorch model to be trained.
        train_loader: DataLoader for the training data.
        val_loader: DataLoader for the validation data.
        epochs: The number of epochs to train the model.
        criterion: The loss function used to evaluate the model's performance.
        optimizer: The optimization algorithm used to update the model's weights.
        device: The device (CPU or GPU) to train the model on.
        early_stopping: An optional instance of CustomEarlyStopping to stop training early if validation loss does not improve.

    Returns:
        The trained model with the best validation loss if early stopping is used, otherwise the model after the last training epoch.
    """
    # Iterate over the entire dataset for a specified number of epochs
    for epoch in range(epochs):
        # Set the model to training mode
        model.train()
        running_loss = 0.0

        # Iterate over the training data
        for inputs, labels in train_loader:

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass: compute the model output
            outputs = model(inputs)
            # Compute the loss
            loss = criterion(outputs, labels)

            # Backward pass: compute the gradient of the loss with respect to model parameters
            loss.backward()
            # Perform a single optimization step to update the model parameters
            optimizer.step()

            # Accumulate the training loss
            running_loss += loss.item()

        # Set the model to evaluation mode
        model.eval()
        val_loss = 0.0
        # Disable gradient calculation for validation to save memory and computations
        with torch.no_grad():
            # Iterate over the validation data
            for inputs, labels in val_loader:
                # Compute the model output
                outputs = model(inputs)
                # Compute the loss
                loss = criterion(outputs, labels)
                # Accumulate the validation loss
                val_loss += loss.item()

        # Print the training and validation loss statistics
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

        # Early Stopping check
        if early_stopping is not None:
            # Call the early stopping logic
            early_stopping(val_loss/len(val_loader), model)
            if early_stopping.early_stop:
                # If early stop condition was met, print a message and break the training loop
                print("Early stopping")
                # Load the best model state (with the lowest validation loss)
                model.load_state_dict(early_stopping.best_state)
                break

    # Return the trained model
    return model

In [61]:
model = train_model(model, dl_train, dl_val, epochs, criterion, optimizer, early_stopping)

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout, bidirectional, rnn_class):
        super().__init__()
        # Verificar que rnn_class es una de las clases de RNN permitidas
        if rnn_class not in [nn.LSTM, nn.GRU, nn.RNN]:
            raise ValueError("rnn_class must be nn.LSTM, nn.GRU, or nn.RNN")

        # Inicializar la capa recurrente con la clase pasada como parámetro
        self.rnn = rnn_class(input_size=input_size,
                             hidden_size=hidden_size,
                             num_layers=num_layers,
                             batch_first=True,
                             bidirectional=bidirectional)
        
        # Calcular la dimensión de salida de la capa recurrente
        # Es doble si es bidireccional
        factor = 2 if bidirectional else 1
        
        self.dropout = nn.Dropout(dropout)
        self.linear1 = nn.Linear(hidden_size * factor, 64)
        self.linear2 = nn.Linear(64, 32)
        self.output_linear = nn.Linear(32, output_size)
        
    def forward(self, x):
        # LSTM devuelve (output, (h_n, c_n)) mientras que GRU y RNN solo devuelve (output, h_n)
        x, _ = self.rnn(x)
        x = self.dropout(x)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.output_linear(x)
        return x
